<a href="https://colab.research.google.com/github/mpintoponce/python_utec/blob/main/Modulo02%3AEstructuras_de_datos_secuenciales/07.Pandas_Actividad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad 41: Caso Trackus

## [0] Librerías necesarias para el proyecto

In [1]:
# Conectamos Google Drive con Google Colab
from google.colab import drive
drive.mount('/gdrive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# Librerías necesarias para el desarrollo del proyecto
import pandas as pd
import numpy as np
#from datetime import date

## [1] Entendimiento de los datos

Etapa inicial en un proyecto de Ciencia de Datos, nos permite explorar las características de los datos a ser modelados.

In [ ]:
# Lectura de los datos (formato parquet)
ruta = "/gdrive/My Drive/[UTEC_2025]/[Programa_Python_Mayo]/ALUMNOS/"
dataset = "dataset_test.parquet"
data = pd.read_parquet(ruta+dataset, engine='pyarrow')

In [ ]:
# Dimensionalidad de los datos a nivel de número de filas y columnas
data.shape

In [ ]:
# Inspección de los datos, cinco registros iniciales
data.head(5)

In [ ]:
data[['Quantity']].describe()

In [ ]:
# Cantidad de clientes únicos
data.Account_id.nunique()

In [ ]:
# Cantidad de productos únicos
data.Product_id.nunique()

In [ ]:
# Cantidad de categorías de productos únicos
data.Category.nunique()

In [ ]:
# Evaluamos a las categorías
data.groupby(['Category'])['Account_id'].count()

In [ ]:
data.Category.unique()

In [ ]:
# Inspeccionamos la compra del cliente (código 33236916)
data[data['Account_id']==33236916]

In [ ]:
# Creamos una lista de campos que serán analizados
vars = data.columns.to_list()
vars

In [ ]:
# Agrupamos la data (está expresada a nivel transaccional) a nivel cliente - producto - día
data = data.groupby(vars[:-1])[vars[-1]].sum().reset_index()

In [ ]:
# Inspeccionamos la compra del cliente (código 33236916)
data[data['Account_id']==33236916]

In [ ]:
# Inspeccionamos la nueva dimensionalidad de los datos
data.shape

In [ ]:
# Inspeccionamos el tipo del dato
data.dtypes

In [ ]:
# Cambiamos el campo Date de objeto a fecha
data['Date'] = data['Date'].astype('datetime64[ns]')

In [ ]:
# Calculamos la fecha mínima y máxima de transacción
print("Fecha Mínima: ", data.Date.min())
print("Fecha Máxima: ", data.Date.max())
print("*"*100)

In [ ]:
# Generamos el resumen de la cantidad de clientes a nivel día
res_date = data.groupby(['Date'])['Account_id'].count().reset_index()

In [ ]:
res_date.head(10)

In [ ]:
# Estadísticas del resumen de la cantidad de clientes a nivel día
res_date['Account_id'].describe().T

In [ ]:
# Calculamos las fechas que presentan un comportamiento atípico a nivel de clientes
p75 = np.quantile(res_date['Account_id'],0.75)
p25 = np.quantile(res_date['Account_id'],0.25)
ric = p75 - p25
l_inf = p25 - (1.5*ric)
l_sup = p75 - (1.5*ric)

In [ ]:
# Calculamos la fecha mínima y máxima de transacción
print("Umbral mínimo de clientes por día: ", l_inf)
print("Umbral máximo de clientes por día ", l_sup)
print("*"*100)

In [ ]:
# Extraemos los campos Year y Month desde el campo Date
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month

In [ ]:
# Inspeccionamos los primeros cinco registros
data.head(5)

In [ ]:
# Guardamos los datos agrupados a nivel cliente - producto - día
ruta = "/gdrive/My Drive/[UTEC_2025]/[Programa_Python_Mayo]/ALUMNOS/out/"
dataset = "data_agrupada.csv"
data.to_csv(ruta+dataset,header='True',index_label='False')

## [2] Preparación de los datos

Etapa en la que limpiamos los datos para simplificar la complejidad natural de la búsqueda de patrones por parte de los algoritmos.

In [ ]:
# Calculamos los rangos intercuartílicos para la identificación de outliers a nivel de cantidad de compras

q3 = np.quantile(data['Quantity'],0.75)
q1 = np.quantile(data['Quantity'],0.25)
ric = q3 - q1
lim_sup = q3 + (1.5*ric)
lim_inf = q1 - (1.5*ric)

In [ ]:
# Límites según criterio de Boxplot
lim_inf, lim_sup

In [ ]:
# Filtramos los casos que se encuentran contenidos entre los límites
data = data[(data['Quantity']>= lim_inf) & (data['Quantity']<= lim_sup)]

In [ ]:
# Exploramos la nueva dimensionalidad de los datos, no están considerados los valores outliers
data.shape

In [ ]:
# Analizamos la cantidad promedio de productos comprados por día
data.groupby(['Month'])['Quantity'].mean()

In [ ]:
# Analizamos la mediana de la cantidad de productos comprados por día
data.groupby(['Month'])['Quantity'].median()

In [ ]:
# Analizamos la dispersión de la cantidad de productos comprados por día
data.groupby(['Month'])['Quantity'].std()